<h1 style="font-size:40px;">Proyecto SQL</h1>

<h1 style="font-size:25px;">Descripción del poyecto: </h1> 

Durante la pandemia, el comportamiento de los consumidores cambió significativamente, con más personas optando por actividades dentro del hogar, como la lectura. Esto generó oportunidades para startups interesadas en desarrollar aplicaciones para lectores. Con base en este contexto, el objetivo del proyecto es analizar una base de datos de un servicio digital de libros que contiene información sobre publicaciones, autores, editoriales, calificaciones y reseñas de usuarios.

Este estudio busca extraer información relevante a través de consultas SQL que permita entender mejor el comportamiento de los usuarios y las características del contenido más valorado. A partir de estos análisis, se podrá generar una propuesta de valor para el desarrollo de un nuevo producto o funcionalidad que compita en el mercado de aplicaciones para lectores.


Los principales objetivos de este analisis son: identificar la cantidad de libros modernos (publicados después del 2000), analizar la relación entre calificaciones y reseñas por libro, determinar qué editorial publica más libros de contenido sustancial (más de 50 páginas), detectar al autor con mejor desempeño en términos de calificaciones, considerando libros con suficiente retroalimentación y evaluar el nivel de participación de los usuarios más activos, mediante la cantidad de reseñas de texto.

In [31]:
import pandas as pd
from sqlalchemy import create_engine


db_config = {
    'user': 'practicum_student',
    'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7',
    'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
    'port': 5432,
    'db': 'data-analyst-final-project-db'
}


connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)


engine = create_engine(connection_string, connect_args={'sslmode':'require'})

<h1 style="font-size:20px;"> 1.-Imprimir las tablas </h1> 

In [32]:
# Tabla: books
query_books = 'SELECT * FROM books LIMIT 5;'
books_sample = pd.read_sql(query_books, con=engine)
print(" Tabla: books")
print(books_sample, '\n')

# Tabla: authors
query_authors = 'SELECT * FROM authors LIMIT 5;'
authors_sample = pd.read_sql(query_authors, con=engine)
print(" Tabla: authors")
print(authors_sample, '\n')

# Tabla: publishers
query_publishers = 'SELECT * FROM publishers LIMIT 5;'
publishers_sample = pd.read_sql(query_publishers, con=engine)
print(" Tabla: publishers")
print(publishers_sample, '\n')

# Tabla: ratings
query_ratings = 'SELECT * FROM ratings LIMIT 5;'
ratings_sample = pd.read_sql(query_ratings, con=engine)
print(" Tabla: ratings")
print(ratings_sample, '\n')

# Tabla: reviews
query_reviews = 'SELECT * FROM reviews LIMIT 5;'
reviews_sample = pd.read_sql(query_reviews, con=engine)
print(" Tabla: reviews")
print(reviews_sample, '\n')

 Tabla: books
   book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125                                               1776   

   num_pages publication_date  publisher_id  
0        594       2005-11-01            93  
1        992       2003-05-22           336  
2        322       2010-12-21           135  
3        541       2006-10-10           309  
4        386       2006-07-04           268   

 Tabla: authors
   author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4                   Alan Brennert
4         

<h1 style="font-size:20px;"> 2.-Número de libros publicados después del 1 de enero de 2000</h1>

In [33]:
query_1 = '''
SELECT COUNT(*) AS books_after_2000
FROM books
WHERE publication_date > '2000-01-01';
'''


result_1 = pd.read_sql(query_1, con=engine)
print(result_1)

   books_after_2000
0               819


<h1 style="font-size:20px;"> 3.-Número de reseñas de usuarios y la calificación promedio para cada libro.</h1>

In [34]:
query_2 = '''
SELECT 
    b.book_id,
    b.title,
    COUNT(DISTINCT r.review_id) AS num_reviews,
    ROUND(AVG(rt.rating), 2) AS avg_rating
FROM books b
LEFT JOIN reviews r ON b.book_id = r.book_id
LEFT JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY b.book_id, b.title
ORDER BY num_reviews DESC;
'''


result_2 = pd.read_sql(query_2, con=engine)
print(result_2.head(10))  

   book_id                                              title  num_reviews  \
0      948                            Twilight (Twilight  #1)            7   
1      963                                Water for Elephants            6   
2      734                                   The Glass Castle            6   
3      302  Harry Potter and the Prisoner of Azkaban (Harr...            6   
4      695  The Curious Incident of the Dog in the Night-Time            6   
5      696             The Da Vinci Code (Robert Langdon  #2)            6   
6      627                                      The Alchemist            6   
7      750                The Hobbit  or There and Back Again            6   
8      656                                     The Book Thief            6   
9      779  The Lightning Thief (Percy Jackson and the Oly...            6   

   avg_rating  
0        3.66  
1        3.98  
2        4.21  
3        4.41  
4        4.08  
5        3.83  
6        3.79  
7        4.13

<h1 style="font-size:20px;"> 4.-Editorial que ha publicado el mayor número de libros con más de 50 páginas.</h1>

In [35]:
query_3 = '''
SELECT 
    p.publisher,
    COUNT(b.book_id) AS num_books
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY num_books DESC
LIMIT 1;
'''


result_3 = pd.read_sql(query_3, con=engine)
print(result_3)

       publisher  num_books
0  Penguin Books         42


<h1 style="font-size:20px;">5.-Autor que tiene la más alta calificación promedio del libro.</h1>

In [36]:
query_4 = '''
WITH book_rating_stats AS (
    SELECT 
        b.book_id,
        b.author_id,
        COUNT(r.rating_id) AS num_ratings,
        AVG(r.rating) AS avg_rating
    FROM books b
    JOIN ratings r ON b.book_id = r.book_id
    GROUP BY b.book_id, b.author_id
    HAVING COUNT(r.rating_id) >= 50
),
author_avg_ratings AS (
    SELECT 
        a.author,
        ROUND(AVG(brs.avg_rating), 2) AS avg_author_rating
    FROM book_rating_stats brs
    JOIN authors a ON brs.author_id = a.author_id
    GROUP BY a.author
)

SELECT 
    author,
    avg_author_rating
FROM author_avg_ratings
ORDER BY avg_author_rating DESC
LIMIT 1;
'''


result_4 = pd.read_sql(query_4, con=engine)
print(result_4)

                       author  avg_author_rating
0  J.K. Rowling/Mary GrandPré               4.28


<h1 style="font-size:20px;"> 6.-Número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.</h1>

In [37]:
query_5 = '''
WITH active_raters AS (
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id) > 50
),
user_reviews AS (
    SELECT ar.username, COUNT(rv.review_id) AS num_reviews
    FROM active_raters ar
    LEFT JOIN reviews rv ON ar.username = rv.username
    GROUP BY ar.username
)

SELECT ROUND(AVG(num_reviews), 2) AS avg_reviews_per_user
FROM user_reviews;
'''


result_5 = pd.read_sql(query_5, con=engine)
print(result_5)

   avg_reviews_per_user
0                 24.33


<h1 style="font-size:35px;"> Conclusiones generales</h1>

Despues de haber realizado la consulta anterior, se puede observar lo siguiente:


-Se han publicado 819 libros publicados después del 1 de enero de 2000 lo cual muestra que si ha habido un numero considerable de publicaciones posterior a la pandemia y refleja el gusto de las personas por leer y aprovechar el tiempo en casa.

-Entre los libros con mayor numero de reseñas y calificacion promedio mas alto de libros son Twilight con  7 reseñasy una calificación promedio 3.66 y Harry Potter and the Prisoner of Azkaban con 6 reseñas y una calificaíon promedio de 4.41, por lo que se puede notar que los libros populares y de literatura juvenil o fantástica suelen tener más reseñas y buenas calificaciones, además de  indica un alta preferencia por parte de los lectores.

-La editorial Penguin Books destaca como la editorial con mayor número de publicaciones en el catálogo (excluyendo folletos u obras breves), por lo que podría ser una editorial clave para establecer alianzas o destacar contenido de calidad en la plataforma.


-La autora con la calificación promedio más alta es J.K. Rowling/Mary GrandPré con 4.28 (que es la autora de la saga de Harry Potter) por lo que se puede notar que no solo son leídos muy leidos y buscados, sino también consistentemente bien valorados, lo que confirma nuevamente su relevancia para el público lector como en sus reseñas y puntuación. Lo anterior nos ayuda a identificar que si se hace alguna alianza con la autora para lanzar un libro nuevo, se tiene más certeza de que el libro tenga exito.

-Por ultimo, el número promedio de reseñas de texto entre usuarios que calificaron más de 50 libros es de 24.33 reseñas por usuario, por lo que se puede notar que los usuarios más activos no solo califican muchos libros, sino que también escriben reseñas detalladas. Al tener usuarios participativos, se podrían implementar nuevas formás para crear más  interacción mediante una nueva aplicación, como foros, recomendaciones o clubes de lectura.


Como comentario final y como respuesta a cual sería una buena opción para generar una propuesta de valor para nuevo producto, se podría hacer alianza con la editorial o editoriales con más participacion en el mercado como lo es Penguin, trabajando en conjunto con autores de renombre internacional para tener más certeza de exito como lo es la autora J.K. Rowling y la ilustradora Mary GrandPré para crear un nuevo libro con temá de interes actual como fantasia. Si se ponen en conjunto los 3 aspectos principales (editorial, autora y genero más buscado) se puede tener más certeza de que el libro puede tener mayor exito en su lanzamiento.